In [2]:
import pickle

from dataset import NewsDataset
from model import DistilBertForSequenceClassification

from smooth_gradient import SmoothGradient
from integrated_gradient import IntegratedGradient

from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)

import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import DistilBertConfig, DistilBertTokenizer

from IPython.display import display, HTML

In [6]:
config = AutoConfig.from_pretrained("bert-base-cased", num_labels=93)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForTokenClassification.from_pretrained("bert-base-cased",
            from_tf=bool(".ckpt" in "bert-base-cased"),
            config=config)

criterion = nn.CrossEntropyLoss()
batch_size = 1

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [9]:
test_example = [
        ["Interpretation of HuggingFase's model decision"], 
        ["Transformer-based models have taken a leading role "
        "in NLP today."]
]

test_dataset = NewsDataset(
    data_list=test_example,
    tokenizer=tokenizer,
    max_length=config.max_position_embeddings, 
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
)
integrated_grad = IntegratedGradient(
    model, 
    criterion, 
    tokenizer, 
    show_progress=True,
    encoder="bert"
)
smooth_grad = SmoothGradient(
    model,
    criterion,
    tokenizer,
    show_progrss=True,
    encoder="bert"
)

# instances = integrated_grad.saliency_interpret(test_dataloader)
instances = smooth_grad.saliency_interpret(test_dataloader)
coloder_string = integrated_grad.colorize(instances[0])

100%|██████████| 1/1 [00:13<00:00, 13.43s/it]


In [10]:
display(HTML(coloder_string))